### Лабораторная работа №6

#### <span style="color:blue">Задание.</span> Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

Сначала мы импортируем необходимые библиотеки и нормализуем данные:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score


data = pd.read_csv("adult.data")
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
                'hours-per-week', 'native-country', 'income']
data.columns = column_names

data = data.replace(' ?', pd.NA)

data = data.dropna()
data = pd.get_dummies(data, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])
data['income'] = data['income'].map({' <=50K': 0, ' >50K': 1})

Разделение данных на обучающий и тестовый наборы

In [2]:
X = data.drop('income', axis=1)
y = data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Метод CatBoost

In [4]:
catboost_model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='Logloss', cat_features=[1, 3, 5, 6, 7, 8, 9, 13])
param_grid_catboost = {
    'iterations': [50, 100, 150],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.2],
    'cat_features': [[1, 3, 5, 6, 7, 8, 9, 13]]
}

grid_search_catboost = GridSearchCV(CatBoostClassifier(), param_grid_catboost, cv=5)

grid_search_catboost.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20, verbose=10)

best_params_catboost = grid_search_catboost.best_params_
print("Лучшие параметры для CatBoost: ", best_params_catboost)

0:	learn: 0.6833220	test: 0.6832981	best: 0.6832981 (0)	total: 167ms	remaining: 8.2s
10:	learn: 0.5973881	test: 0.5974540	best: 0.5974540 (10)	total: 361ms	remaining: 1.28s
20:	learn: 0.5342732	test: 0.5346322	best: 0.5346322 (20)	total: 554ms	remaining: 765ms
30:	learn: 0.4887083	test: 0.4894535	best: 0.4894535 (30)	total: 740ms	remaining: 453ms
40:	learn: 0.4531118	test: 0.4544995	best: 0.4544995 (40)	total: 925ms	remaining: 203ms
49:	learn: 0.4286268	test: 0.4306802	best: 0.4306802 (49)	total: 1.09s	remaining: 0us

bestTest = 0.4306801786
bestIteration = 49

0:	learn: 0.6835258	test: 0.6835378	best: 0.6835378 (0)	total: 20.4ms	remaining: 1s
10:	learn: 0.6004627	test: 0.6004284	best: 0.6004284 (10)	total: 199ms	remaining: 706ms
20:	learn: 0.5380518	test: 0.5383032	best: 0.5383032 (20)	total: 373ms	remaining: 516ms
30:	learn: 0.4902266	test: 0.4908322	best: 0.4908322 (30)	total: 554ms	remaining: 339ms
40:	learn: 0.4558274	test: 0.4568656	best: 0.4568656 (40)	total: 729ms	remaining: 16

40:	learn: 0.2890809	test: 0.2973964	best: 0.2973964 (40)	total: 797ms	remaining: 175ms
49:	learn: 0.2861148	test: 0.2953099	best: 0.2953099 (49)	total: 968ms	remaining: 0us

bestTest = 0.295309946
bestIteration = 49

0:	learn: 0.6833220	test: 0.6832981	best: 0.6832981 (0)	total: 19.1ms	remaining: 1.89s
10:	learn: 0.5973881	test: 0.5974540	best: 0.5974540 (10)	total: 206ms	remaining: 1.67s
20:	learn: 0.5342732	test: 0.5346322	best: 0.5346322 (20)	total: 390ms	remaining: 1.47s
30:	learn: 0.4887083	test: 0.4894535	best: 0.4894535 (30)	total: 565ms	remaining: 1.26s
40:	learn: 0.4531118	test: 0.4544995	best: 0.4544995 (40)	total: 752ms	remaining: 1.08s
50:	learn: 0.4263327	test: 0.4283860	best: 0.4283860 (50)	total: 933ms	remaining: 897ms
60:	learn: 0.4085293	test: 0.4107466	best: 0.4107466 (60)	total: 1.11s	remaining: 712ms
70:	learn: 0.3938861	test: 0.3964340	best: 0.3964340 (70)	total: 1.3s	remaining: 531ms
80:	learn: 0.3831872	test: 0.3858297	best: 0.3858297 (80)	total: 1.49s	remaining

0:	learn: 0.5966873	test: 0.5976640	best: 0.5976640 (0)	total: 21.6ms	remaining: 2.14s
10:	learn: 0.3584543	test: 0.3606269	best: 0.3606269 (10)	total: 223ms	remaining: 1.8s
20:	learn: 0.3222599	test: 0.3265025	best: 0.3265025 (20)	total: 426ms	remaining: 1.6s
30:	learn: 0.3109225	test: 0.3158770	best: 0.3158770 (30)	total: 626ms	remaining: 1.39s
40:	learn: 0.3033382	test: 0.3080544	best: 0.3080544 (40)	total: 843ms	remaining: 1.21s
50:	learn: 0.2985825	test: 0.3038823	best: 0.3038823 (50)	total: 1.06s	remaining: 1.02s
60:	learn: 0.2956760	test: 0.3021982	best: 0.3021982 (60)	total: 1.27s	remaining: 814ms
70:	learn: 0.2918016	test: 0.2989335	best: 0.2989335 (70)	total: 1.49s	remaining: 610ms
80:	learn: 0.2896695	test: 0.2971108	best: 0.2971108 (80)	total: 1.72s	remaining: 402ms
90:	learn: 0.2874434	test: 0.2953581	best: 0.2953581 (90)	total: 1.91s	remaining: 189ms
99:	learn: 0.2852144	test: 0.2935469	best: 0.2935469 (99)	total: 2.1s	remaining: 0us

bestTest = 0.2935469261
bestIteration

130:	learn: 0.3487175	test: 0.3521230	best: 0.3521230 (130)	total: 2.75s	remaining: 398ms
140:	learn: 0.3446283	test: 0.3482675	best: 0.3482675 (140)	total: 2.97s	remaining: 190ms
149:	learn: 0.3410982	test: 0.3449516	best: 0.3449516 (149)	total: 3.18s	remaining: 0us

bestTest = 0.3449515706
bestIteration = 149

0:	learn: 0.6835258	test: 0.6835378	best: 0.6835378 (0)	total: 21.1ms	remaining: 3.14s
10:	learn: 0.6004627	test: 0.6004284	best: 0.6004284 (10)	total: 208ms	remaining: 2.63s
20:	learn: 0.5380518	test: 0.5383032	best: 0.5383032 (20)	total: 393ms	remaining: 2.41s
30:	learn: 0.4902266	test: 0.4908322	best: 0.4908322 (30)	total: 588ms	remaining: 2.25s
40:	learn: 0.4558274	test: 0.4568656	best: 0.4568656 (40)	total: 778ms	remaining: 2.07s
50:	learn: 0.4313469	test: 0.4328577	best: 0.4328577 (50)	total: 976ms	remaining: 1.89s
60:	learn: 0.4124927	test: 0.4142538	best: 0.4142538 (60)	total: 1.17s	remaining: 1.71s
70:	learn: 0.3974071	test: 0.3994825	best: 0.3994825 (70)	total: 1.36s	

80:	learn: 0.2918962	test: 0.2991905	best: 0.2991905 (80)	total: 1.47s	remaining: 1.25s
90:	learn: 0.2900080	test: 0.2974625	best: 0.2974625 (90)	total: 1.66s	remaining: 1.08s
100:	learn: 0.2875176	test: 0.2954616	best: 0.2954616 (100)	total: 1.86s	remaining: 901ms
110:	learn: 0.2848160	test: 0.2932762	best: 0.2932762 (110)	total: 2.04s	remaining: 718ms
120:	learn: 0.2828887	test: 0.2922748	best: 0.2922748 (120)	total: 2.23s	remaining: 534ms
130:	learn: 0.2811548	test: 0.2916202	best: 0.2916202 (130)	total: 2.43s	remaining: 353ms
140:	learn: 0.2796431	test: 0.2909199	best: 0.2909199 (140)	total: 2.63s	remaining: 168ms
149:	learn: 0.2784431	test: 0.2904387	best: 0.2904387 (149)	total: 2.81s	remaining: 0us

bestTest = 0.2904387048
bestIteration = 149

0:	learn: 0.6037356	test: 0.6035888	best: 0.6035888 (0)	total: 22.4ms	remaining: 3.33s
10:	learn: 0.3560001	test: 0.3587074	best: 0.3587074 (10)	total: 220ms	remaining: 2.77s
20:	learn: 0.3238823	test: 0.3293146	best: 0.3293146 (20)	total: 

...

30:	learn: 0.2735561	test: 0.2931005	best: 0.2931005 (30)	total: 951ms	remaining: 583ms
40:	learn: 0.2665329	test: 0.2900412	best: 0.2898720 (39)	total: 1.25s	remaining: 274ms
49:	learn: 0.2618743	test: 0.2892772	best: 0.2892772 (49)	total: 1.54s	remaining: 0us

bestTest = 0.2892772327
bestIteration = 49

0:	learn: 0.6822178	test: 0.6823026	best: 0.6823026 (0)	total: 31.3ms	remaining: 3.1s
10:	learn: 0.5865976	test: 0.5874681	best: 0.5874681 (10)	total: 334ms	remaining: 2.7s
20:	learn: 0.5197535	test: 0.5207889	best: 0.5207889 (20)	total: 624ms	remaining: 2.35s
30:	learn: 0.4680070	test: 0.4698469	best: 0.4698469 (30)	total: 943ms	remaining: 2.1s
40:	learn: 0.4317277	test: 0.4341227	best: 0.4341227 (40)	total: 1.23s	remaining: 1.77s
50:	learn: 0.4044227	test: 0.4074954	best: 0.4074954 (50)	total: 1.56s	remaining: 1.5s
60:	learn: 0.3832165	test: 0.3867561	best: 0.3867561 (60)	total: 1.91s	remaining: 1.22s
70:	learn: 0.3668427	test: 0.3708158	best: 0.3708158 (70)	total: 2.23s	remaining: 

99:	learn: 0.2596072	test: 0.2871820	best: 0.2871602 (98)	total: 3.05s	remaining: 0us

bestTest = 0.2871602193
bestIteration = 98

Shrink model to first 99 iterations.
0:	learn: 0.5952291	test: 0.5959119	best: 0.5959119 (0)	total: 25.2ms	remaining: 2.49s
10:	learn: 0.3297159	test: 0.3355827	best: 0.3355827 (10)	total: 338ms	remaining: 2.73s
20:	learn: 0.2994118	test: 0.3087160	best: 0.3087160 (20)	total: 647ms	remaining: 2.44s
30:	learn: 0.2895392	test: 0.3011756	best: 0.3011756 (30)	total: 961ms	remaining: 2.14s
40:	learn: 0.2826620	test: 0.2980512	best: 0.2980512 (40)	total: 1.27s	remaining: 1.83s
50:	learn: 0.2772079	test: 0.2950052	best: 0.2950052 (50)	total: 1.57s	remaining: 1.51s
60:	learn: 0.2733345	test: 0.2936423	best: 0.2936423 (60)	total: 1.88s	remaining: 1.2s
70:	learn: 0.2681298	test: 0.2916422	best: 0.2916422 (70)	total: 2.18s	remaining: 891ms
80:	learn: 0.2653586	test: 0.2906005	best: 0.2906005 (80)	total: 2.47s	remaining: 579ms
90:	learn: 0.2619841	test: 0.2896143	best:

130:	learn: 0.3212649	test: 0.3282863	best: 0.3282863 (130)	total: 4.14s	remaining: 601ms
140:	learn: 0.3175734	test: 0.3250571	best: 0.3250571 (140)	total: 4.5s	remaining: 287ms
149:	learn: 0.3144039	test: 0.3223030	best: 0.3223030 (149)	total: 4.79s	remaining: 0us

bestTest = 0.3223030023
bestIteration = 149

0:	learn: 0.6821365	test: 0.6822730	best: 0.6822730 (0)	total: 33ms	remaining: 4.92s
10:	learn: 0.5901402	test: 0.5906558	best: 0.5906558 (10)	total: 323ms	remaining: 4.08s
20:	learn: 0.5207982	test: 0.5216116	best: 0.5216116 (20)	total: 681ms	remaining: 4.18s
30:	learn: 0.4727670	test: 0.4739421	best: 0.4739421 (30)	total: 977ms	remaining: 3.75s
40:	learn: 0.4345852	test: 0.4364417	best: 0.4364417 (40)	total: 1.28s	remaining: 3.41s
50:	learn: 0.4060422	test: 0.4083851	best: 0.4083851 (50)	total: 1.59s	remaining: 3.1s
60:	learn: 0.3854698	test: 0.3883548	best: 0.3883548 (60)	total: 1.9s	remaining: 2.78s
70:	learn: 0.3701473	test: 0.3733430	best: 0.3733430 (70)	total: 2.19s	remai

70:	learn: 0.2699439	test: 0.2913388	best: 0.2913388 (70)	total: 2.36s	remaining: 2.63s
80:	learn: 0.2665956	test: 0.2911864	best: 0.2911381 (78)	total: 2.68s	remaining: 2.28s
90:	learn: 0.2633205	test: 0.2906274	best: 0.2905736 (88)	total: 3s	remaining: 1.95s
100:	learn: 0.2599885	test: 0.2888684	best: 0.2888684 (100)	total: 3.33s	remaining: 1.62s
110:	learn: 0.2559622	test: 0.2891943	best: 0.2888684 (100)	total: 3.66s	remaining: 1.28s
120:	learn: 0.2523487	test: 0.2880504	best: 0.2880504 (120)	total: 4s	remaining: 957ms
130:	learn: 0.2498083	test: 0.2878803	best: 0.2878803 (130)	total: 4.32s	remaining: 626ms
140:	learn: 0.2468953	test: 0.2875956	best: 0.2875890 (139)	total: 4.64s	remaining: 297ms
149:	learn: 0.2445163	test: 0.2872616	best: 0.2872616 (149)	total: 4.93s	remaining: 0us

bestTest = 0.2872615711
bestIteration = 149

0:	learn: 0.5928789	test: 0.5937131	best: 0.5937131 (0)	total: 35.5ms	remaining: 5.29s
10:	learn: 0.3279595	test: 0.3341000	best: 0.3341000 (10)	total: 367ms	

Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.2898563143
bestIteration = 67

Shrink model to first 68 iterations.
0:	learn: 0.5190015	test: 0.5203037	best: 0.5203037 (0)	total: 26.2ms	remaining: 3.91s
10:	learn: 0.2980832	test: 0.3072665	best: 0.3072665 (10)	total: 359ms	remaining: 4.54s
20:	learn: 0.2831950	test: 0.2971258	best: 0.2971258 (20)	total: 690ms	remaining: 4.24s
30:	learn: 0.2737782	test: 0.2933797	best: 0.2933797 (30)	total: 1.02s	remaining: 3.92s
40:	learn: 0.2669421	test: 0.2912908	best: 0.2908536 (38)	total: 1.35s	remaining: 3.59s
50:	learn: 0.2602531	test: 0.2887161	best: 0.2887161 (50)	total: 1.68s	remaining: 3.26s
60:	learn: 0.2538038	test: 0.2875629	best: 0.2875629 (60)	total: 2.02s	remaining: 2.95s
70:	learn: 0.2479226	test: 0.2877994	best: 0.2873774 (68)	total: 2.36s	remaining: 2.63s
80:	learn: 0.2437821	test: 0.2874936	best: 0.2872547 (76)	total: 2.69s	remaining: 2.29s
90:	learn: 0.2391327	test: 0.2877145	best: 0.2872547 (76)	total: 3.02s	re

In [5]:
best_catboost_model = CatBoostClassifier(iterations=best_params_catboost['iterations'],
                                         depth=best_params_catboost['depth'],
                                         learning_rate=best_params_catboost['learning_rate'],
                                         cat_features=best_params_catboost['cat_features'])

best_catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20, verbose=10)

y_pred_best_catboost = best_catboost_model.predict(X_test)

accuracy_best_catboost = accuracy_score(y_test, y_pred_best_catboost)
report_best_catboost = classification_report(y_test, y_pred_best_catboost)

print(f'Точность CatBoost: {accuracy_best_catboost}')
print('Отчет о классификации CatBoost:\n', report_best_catboost)

0:	learn: 0.5083358	test: 0.5123636	best: 0.5123636 (0)	total: 39.3ms	remaining: 3.89s
10:	learn: 0.2971496	test: 0.3081134	best: 0.3081134 (10)	total: 391ms	remaining: 3.16s
20:	learn: 0.2824385	test: 0.2962378	best: 0.2962378 (20)	total: 698ms	remaining: 2.63s
30:	learn: 0.2735952	test: 0.2922307	best: 0.2922307 (30)	total: 1.01s	remaining: 2.24s
40:	learn: 0.2673472	test: 0.2901839	best: 0.2901839 (40)	total: 1.31s	remaining: 1.89s
50:	learn: 0.2608540	test: 0.2883342	best: 0.2883342 (50)	total: 1.65s	remaining: 1.58s
60:	learn: 0.2551065	test: 0.2875646	best: 0.2874440 (57)	total: 2s	remaining: 1.28s
70:	learn: 0.2499184	test: 0.2870431	best: 0.2869044 (69)	total: 2.35s	remaining: 959ms
80:	learn: 0.2457177	test: 0.2874628	best: 0.2868886 (72)	total: 2.69s	remaining: 631ms
90:	learn: 0.2417602	test: 0.2876253	best: 0.2868886 (72)	total: 3.08s	remaining: 305ms
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.2868885842
bestIteration = 72

Shrink model to first 73 

Выбор гиперпараметров CatBoost:
- iterations: Количество итераций (деревьев) для обучения. Увеличение числа итераций может улучшить производительность, но требует больше времени для обучения.
- depth: Максимальная глубина дерева. Это помогает избежать переобучения. Мы выбрали относительно большую глубину для более сложной модели.
- learning_rate: Скорость обучения. Определяет величину коррекции весов на каждой итерации. Мы выбрали стандартное значение 0.1.
- loss_function='Logloss': Функция потерь для бинарной классификации.

In [6]:
cv_scores_catboost = cross_val_score(best_catboost_model, X, y, cv=5)

print('Результаты кросс-валидации CatBoost:', cv_scores_catboost)
print(f'Средняя точность (CatBoost): {cv_scores_catboost.mean()}')

0:	learn: 0.5127619	total: 38.6ms	remaining: 3.82s
1:	learn: 0.4227276	total: 74.2ms	remaining: 3.64s
2:	learn: 0.3809470	total: 112ms	remaining: 3.61s
3:	learn: 0.3519259	total: 143ms	remaining: 3.43s
4:	learn: 0.3341696	total: 184ms	remaining: 3.49s
5:	learn: 0.3225536	total: 219ms	remaining: 3.44s
6:	learn: 0.3147347	total: 254ms	remaining: 3.38s
7:	learn: 0.3094704	total: 295ms	remaining: 3.39s
8:	learn: 0.3053107	total: 330ms	remaining: 3.33s
9:	learn: 0.3025553	total: 362ms	remaining: 3.26s
10:	learn: 0.2994774	total: 392ms	remaining: 3.17s
11:	learn: 0.2970704	total: 432ms	remaining: 3.17s
12:	learn: 0.2944817	total: 472ms	remaining: 3.16s
13:	learn: 0.2929215	total: 513ms	remaining: 3.15s
14:	learn: 0.2915068	total: 551ms	remaining: 3.12s
15:	learn: 0.2894526	total: 587ms	remaining: 3.08s
16:	learn: 0.2874043	total: 624ms	remaining: 3.04s
17:	learn: 0.2871626	total: 640ms	remaining: 2.92s
18:	learn: 0.2860517	total: 669ms	remaining: 2.85s
19:	learn: 0.2844805	total: 700ms	remai

64:	learn: 0.2568627	total: 2.12s	remaining: 1.14s
65:	learn: 0.2564288	total: 2.16s	remaining: 1.11s
66:	learn: 0.2561472	total: 2.19s	remaining: 1.08s
67:	learn: 0.2558082	total: 2.22s	remaining: 1.04s
68:	learn: 0.2550135	total: 2.25s	remaining: 1.01s
69:	learn: 0.2544825	total: 2.29s	remaining: 983ms
70:	learn: 0.2541782	total: 2.33s	remaining: 951ms
71:	learn: 0.2538412	total: 2.36s	remaining: 917ms
72:	learn: 0.2533695	total: 2.39s	remaining: 885ms
73:	learn: 0.2529002	total: 2.42s	remaining: 851ms
74:	learn: 0.2524028	total: 2.46s	remaining: 819ms
75:	learn: 0.2517423	total: 2.49s	remaining: 787ms
76:	learn: 0.2513515	total: 2.52s	remaining: 753ms
77:	learn: 0.2505914	total: 2.55s	remaining: 720ms
78:	learn: 0.2503242	total: 2.58s	remaining: 686ms
79:	learn: 0.2499230	total: 2.61s	remaining: 653ms
80:	learn: 0.2495704	total: 2.64s	remaining: 620ms
81:	learn: 0.2488034	total: 2.68s	remaining: 587ms
82:	learn: 0.2483258	total: 2.71s	remaining: 556ms
83:	learn: 0.2479837	total: 2.7

28:	learn: 0.2768465	total: 946ms	remaining: 2.32s
29:	learn: 0.2762468	total: 976ms	remaining: 2.28s
30:	learn: 0.2753029	total: 1s	remaining: 2.24s
31:	learn: 0.2739337	total: 1.04s	remaining: 2.2s
32:	learn: 0.2732605	total: 1.07s	remaining: 2.16s
33:	learn: 0.2726690	total: 1.1s	remaining: 2.13s
34:	learn: 0.2724429	total: 1.12s	remaining: 2.08s
35:	learn: 0.2723663	total: 1.14s	remaining: 2.02s
36:	learn: 0.2719186	total: 1.17s	remaining: 1.98s
37:	learn: 0.2711830	total: 1.2s	remaining: 1.96s
38:	learn: 0.2699160	total: 1.23s	remaining: 1.92s
39:	learn: 0.2695373	total: 1.26s	remaining: 1.89s
40:	learn: 0.2689454	total: 1.28s	remaining: 1.85s
41:	learn: 0.2683841	total: 1.32s	remaining: 1.82s
42:	learn: 0.2681083	total: 1.34s	remaining: 1.78s
43:	learn: 0.2678134	total: 1.38s	remaining: 1.75s
44:	learn: 0.2670568	total: 1.41s	remaining: 1.72s
45:	learn: 0.2664503	total: 1.44s	remaining: 1.7s
46:	learn: 0.2660391	total: 1.48s	remaining: 1.67s
47:	learn: 0.2655668	total: 1.51s	rema

92:	learn: 0.2438080	total: 2.92s	remaining: 220ms
93:	learn: 0.2432208	total: 2.95s	remaining: 188ms
94:	learn: 0.2428152	total: 2.98s	remaining: 157ms
95:	learn: 0.2421146	total: 3.01s	remaining: 125ms
96:	learn: 0.2416053	total: 3.04s	remaining: 94ms
97:	learn: 0.2411958	total: 3.07s	remaining: 62.6ms
98:	learn: 0.2407790	total: 3.1s	remaining: 31.3ms
99:	learn: 0.2404200	total: 3.12s	remaining: 0us
Результаты кросс-валидации CatBoost: [0.86507542 0.8655504  0.86969496 0.86952918 0.87185013]
Средняя точность (CatBoost): 0.8683400173932847
